In [ ]:
#Natural Language processing (NLP) is the field or discipline in computing, or machine learning
#that deals with trying to understand natural human languages, spellcheck, autocomplete, voice assistants, translation
#Recurrent Neural Networks(RNN) is much more capable of processing sequential datasuch as text or characters
#Sequence Data
#Need to convert the text data to numeric

In [ ]:
#Bag of words algorithm: every single word in the vocabulary is going to be placed in a dictionary and have some integer that represents it
#only going to keep track of the words that are present and the frequency of those words, you ose  the order of the words
#this metod lose hte context of the words within the sentence,very different meaning they are encoded exaclty the same way

In [ ]:
#This isn't really the way we would do this in practice, it gives an idea of how bag of words works. 
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)


In [ ]:
#words embedding
#try to find a wat to represent words that are similar using very similar numbers
#it classify or translate every single word into a vector
#very similar vectors are similar words, and opposite are far away
#the word embeddings are actually trained, and learns these word embeddings as it goes, so we can add it layer to give a vector that implies the context of the word in a sentence


In [ ]:
#Recurren neural network (RNN)
#the diference is that it contains an internal loop
#doesnt process the entire input data at once, processes it at different time steps
#mantains an internal memory of an internal state, the new input will be trated based on the previous input and its context
#procces one word at a time, generate output based in it, and save internal memory state that is keeping track to do the calculation
#the output from 1 word will be added to the next input (simple recurrent layer)
#large sentences implies inscreasingly difficult fot the model to actually build a good understanding
#LST (long short term memory, the next layer)
#add another component that keep tracking the internal state
#add all the outputs of that layer, allows use all these values


In [ ]:
#Sentiment analysis
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584
#0 more common, 88584 less common
MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 2s 0us/step


In [ ]:
# Lets look at one review
train_data[1]


In [ ]:
#If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore, we must make each review the same length. 
from keras.utils import pad_sequences
#trim extra words and add the necessary amount of 0's to make it equal to maxlen
train_data = pad_sequences(train_data, MAXLEN)
test_data = pad_sequences(test_data, MAXLEN)  

In [ ]:
#Create the model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32), #embedding
    tf.keras.layers.LSTM(32), #lstm
    tf.keras.layers.Dense(1, activation="sigmoid") #dense with activation
]) #output between 0 and 1

In [ ]:
model.summary()

In [ ]:
#Training

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=7, validation_split=0.2) #use the 0.2 % of the training data to validate the model

Epoch 1/7
625/625 [==============================] - 50s 77ms/step - loss: 0.4040 - acc: 0.8213 - val_loss: 0.2955 - val_acc: 0.8816
Epoch 2/7
625/625 [==============================] - 18s 29ms/step - loss: 0.2466 - acc: 0.9064 - val_loss: 0.2996 - val_acc: 0.8880
Epoch 3/7
625/625 [==============================] - 13s 20ms/step - loss: 0.1944 - acc: 0.9289 - val_loss: 0.3194 - val_acc: 0.8620
Epoch 4/7
625/625 [==============================] - 11s 17ms/step - loss: 0.1570 - acc: 0.9449 - val_loss: 0.3420 - val_acc: 0.8838
Epoch 5/7
625/625 [==============================] - 13s 21ms/step - loss: 0.1321 - acc: 0.9547 - val_loss: 0.3329 - val_acc: 0.8620
Epoch 6/7
625/625 [==============================] - 12s 19ms/step - loss: 0.1081 - acc: 0.9639 - val_loss: 0.3420 - val_acc: 0.8610
Epoch 7/7
625/625 [==============================] - 11s 17ms/step - loss: 0.0920 - acc: 0.9699 - val_loss: 0.5166 - val_acc: 0.8658


In [ ]:
#Evaluate
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 5s 6ms/step - loss: 0.6211 - acc: 0.8428
[0.6211398243904114, 0.8427600264549255]


In [ ]:
#Making predictions
#Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well load the encodings from the dataset and use them to encode our own data.
word_index = imdb.get_word_index()
#check if the words are in our vocabulary

def encode_text(text): #convert the word in tokens
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

In [ ]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250)) #blank
  pred[0] = encoded_text #insert our one entry into this
  result = model.predict(pred)  #model predict that array
  print(result[0]) #

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 [==============================] - 0s 352ms/step
[0.9298646]
1/1 [==============================] - 0s 21ms/step
[0.1962332]


In [1]:
#RNN play generate
#will try to predict next character in the sentences

%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
#import any local text
from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

In [9]:
#Encoding, need to convert the text to numbers
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text]) #every single character to int  and put it on the list

text_as_int = text_to_int(text)

In [10]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [11]:
#function that can convert our numeric values to text
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [15]:
#Create training examples
# smth input: Hell | output: ello

seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) #string data set to char

In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) #take the entire data and batch it into length of 101

In [16]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:

for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))

In [19]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
#Build the model
#pass batches of size 64,  but then this model we be saved and going to use witch batches of size 1 
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]), #none, dont know how long the batch size is going to be
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,#need output from every single time
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size) #amounf of nodes in it rqueal to the amoun of characters in the vocabulary, probability distribution of that character comes next
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

In [ ]:
#pass inputs of 64 entries with sequences of lenght 100 as a training data
#later on we are going to pass entris of 1

In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape
  #every prediction its going to contain 65 numbers, and thats gooing to be the probability of every one of those caracters occurring
  

In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
#every single training example has 100 putputs
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

In [29]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)
#sampling is pick a character based on a probability distribution, it doesn't guarantee that the caracter with the highest probability will be picked, it just uses those probabilities to pick it 
predicted_chars  # and this is what the model predicted for training sequence 1

"YiPRqKGbQo!F&'JKd'!tEQnwD::mKyjPmbZGcDFgqLfKhW: s:VIUOpa P3RmNjzmJ'pl&P?WK,FvJZMWRWn-suc3xV:C:ReFIbx"

In [30]:
#Loss function 
#need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [32]:
#Compile
model.compile(optimizer='adam', loss=loss)

In [33]:
#Creatin checkpoints 
#This will allow us to load our model from a checkpoint and continue training it.
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [35]:
history = model.fit(data, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 14s 68ms/step - loss: 1.4068
Epoch 2/10
172/172 [==============================] - 14s 68ms/step - loss: 1.3524
Epoch 3/10
172/172 [==============================] - 14s 69ms/step - loss: 1.3084
Epoch 4/10
172/172 [==============================] - 14s 69ms/step - loss: 1.2691
Epoch 5/10
172/172 [==============================] - 14s 70ms/step - loss: 1.2321
Epoch 6/10
172/172 [==============================] - 15s 74ms/step - loss: 1.1963
Epoch 7/10
172/172 [==============================] - 15s 71ms/step - loss: 1.1594
Epoch 8/10
172/172 [==============================] - 14s 70ms/step - loss: 1.1236
Epoch 9/10
172/172 [==============================] - 14s 72ms/step - loss: 1.0830
Epoch 10/10
172/172 [==============================] - 16s 71ms/step - loss: 1.0440


In [39]:
#Rebuild the model with batch size 1
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [44]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None])) #Extpect the input 1 and then none dont know what the next input dimension will be

In [ ]:
#Can load any checkpoint we want by specifying the exact file to load.
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

In [45]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [46]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeon,
But ne'er come to my assishance that he cap'tow,
All to thy favour and my mother doth,
With that now I must be condemn'd by consul.

JULIET:
Come, come, tell thee, Signior Baptista:
Why, here's it speak.

Clown:
His bloody days condemned customing modesty joy
To the ot for as this brand when her mad
When he did sind them bows, post he thence,
My friend, my lord, to wed with all my love;
Look to the roof drink butching. Pray you,
The late ye untimely but a little world,
In your commission: let's away my horse.
God save the boyou born to bear wearest: therefore,
' good sooner and to rave hrost
As that evilar earth?
Her sons and peace, it press'd to do set up
The least is merit; yet my father's bosom live we think; who, if you rave med!

RICHARD:
The crding open truth of my pass, when of t
